In [1]:
# from vllm import LLM, SamplingParams
# import json

# model = "/home2/xjw/LLaMA-Factory/models/llama3_lora_sft"
# llm = LLM(model=model, tensor_parallel_size=1)



In [1]:
from vllm import LLM, SamplingParams
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = "/home2/xjw/LLaMA-Factory/models/Qwen1.5-1.8B-Chat"
# lora_path = "/home2/xjw/LLaMA-Factory/saves/Phi3-4B-4k-Chat/lora/train_2024-06-05-21-06-40/checkpoint-90"

tokenizer = AutoTokenizer.from_pretrained(mode_path, local_files_only=True)
# model = AutoModelForCausalLM.from_pretrained(mode_path, local_files_only=True)
# lora_model  = PeftModel.from_pretrained(model, model_id=lora_path)

# llm = LLM(model=lora_model, tensor_parallel_size=1)
llm = LLM(model=mode_path)



/home2/xjw/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-06 18:35:58,601	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-06 18:36:00 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='/home2/xjw/LLaMA-Factory/models/Qwen1.5-1.8B-Chat', speculative_config=None, tokenizer='/home2/xjw/LLaMA-Factory/models/Qwen1.5-1.8B-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/home2/xjw/LLaMA-Factory/models/Qwen1.5-1.8B-Chat)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-06 18:36:03 model_runner.py:146] Loading model weights took 3.4653 GB
INFO 06-06 18:36:04 gpu_executor.py:83] # GPU blocks: 22509, # CPU blocks: 1365
INFO 06-06 18:36:07 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-06 18:36:07 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-06 18:36:13 model_runner.py:924] Graph capturing finished in 6 secs.


In [11]:
def build_prompt(instruction, input, resp1, resp2, result=None, explain=None, ref=None):
    rsp = f"### Response 1:\n{resp1}\n\n### Response 2:\n{resp2}"

    if input:
        input_sequence = f"Below are two responses for a given task. The task is defined by the Instruction with an Input that provides further context. Evaluate the responses and generate a reference answer for the task.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n{rsp}\n\n### Evaluation:\n"
    else:
        input_sequence = f"Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.\n\n### Instruction:\n{instruction}\n\n{rsp}\n\n### Evaluation:\n"


    output_sequence = f"{result}\n\n### Reason: {explain}\n\n### Reference: {ref}\n"
    
    return input_sequence, output_sequence

In [9]:
pandalm_test_set_path = "/home2/xjw/PandaLM/data/testset-v1.json"

pandalm_test_set = []
with open(pandalm_test_set_path, "r") as f:
  pandalm_test_set = json.load(f)

In [12]:
pandalm_test_set_prompt = []
for i in range(len(pandalm_test_set)):
  for j in range(3):
    pandalm_test_set_prompt.append(
      build_prompt(
        instruction=pandalm_test_set[i]['instruction'],
        input=pandalm_test_set[i]['input'] if 'input' in pandalm_test_set[i] else None,
        resp1=pandalm_test_set[i]['response1'],
        resp2=pandalm_test_set[i]['response2'],
        result=pandalm_test_set[i]['annotator' + str(j+1)] ,
      )
    )

In [5]:
prompt_template = """<|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"""
inputs = [ prompt_template.format(input = i[0]) for i in pandalm_test_set_prompt[ : : 3]]


In [6]:
print(inputs[6])
print(len(inputs))

<|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below are two responses for a given task. The task is defined by the Instruction with an Input that provides further context. Evaluate the responses and generate a reference answer for the task.

### Instruction:
Make the article available in a second language by translating it into it.

### Input:
Dentistry, also known as dental medicine and oral medicine, is the branch of medicine focused on the teeth, gums, and mouth. It consists of the study, diagnosis, prevention, management, and treatment of diseases, disorders, and conditions of the mouth, most commonly focused on dentition (the development and arrangement of teeth) as well as the oral mucosa. Dentistry may also encompass other aspects of the craniofacial complex including the temporomandibular joint. The practitioner is called a dentist.
Translate to French:

### Response 1:
La dentisterie, également connu

In [7]:
from vllm.lora.request import LoRARequest

stop_tokens = ["<|end_header_id|>", "<|eot_id|>"]
# sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=1024)
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=1024, stop=stop_tokens)

completions = llm.generate(inputs, sampling_params=sampling_params)

generated_responses = [output.outputs[0].text for output in completions]




Processed prompts: 100%|██████████| 999/999 [00:35<00:00, 27.88it/s, Generation Speed: 3403.34 toks/s]


In [8]:
import json
file_path = "/home2/xjw/output/pandalm_test_Qwen1.5-1.8B-Chat_set_output.json"
with open(file_path, "w") as f:
  json.dump(generated_responses, f)

In [1]:
import json
file_path = "/home2/xjw/output/pandalm_test_Qwen1.5-1.8B-Chat_set_output.json"

with open(file_path, "r") as f:
  generated_responses = json.load(f)




In [2]:
print(len(generated_responses))
# print(inputs[0])
print(generated_responses[0])

# print(inputs[1])
print(generated_responses[1])

# print(inputs[2])
# print(generated_responses[2])

# print(inputs[3])
# print(generated_responses[3])

# print(inputs[4])
print(generated_responses[4])

# print(inputs[5])
print(generated_responses[5])



999
### Reason: Response 1 is better because it is more concise and clear.

### Reference: If you have any questions, please let me know.

### Reason: Both responses are clear and concise, but Response 1 is more concise.

### Reference: If you have any questions about my rate, please let me know.

### Reason: Response 1 is better because it provides a variety of search queries related to coding.

### Reference: 1. What is coding? 2. How to code? 3. How to write code? 4. How to debug code? 5. How to debug code? 6. How to debug code? 7. How to debug code? 8. How to debug code? 9. How to debug code?

### Reason: Response 1 is better because it provides more diverse and relevant search queries.

### Reference: 1. "coding tutorials" 2. "coding languages" 3. "coding languages for beginners" 4. "coding languages for beginners" 5. "coding languages for beginners" 6. "coding languages for beginners" 7. "coding languages for beginners" 8. "coding languages for beginners" 9. "coding languages for

In [3]:
# print(generated_responses[2])

In [7]:
# ans = [ res.split("\n\n")[0] for res in generated_responses]

ans = [ res.split("\n")[0] for res in generated_responses]

newlist = []


# for s in ans:
    
#     if "1" in s or "first" in s:
#         newlist.append("1")
#     elif "2" in s or "second" in s:
#         newlist.append("2")
#     elif "Tie" in s or "both" in s:
#         newlist.append("0")   


# for s in ans:
#     if "Tie" in s:
#         newlist.append("0")
#     elif "1" in s or "first" in s:
#         newlist.append("1")
#     elif "2" in s or "second" in s:
#         newlist.append("2")
#     elif "both" in s:
#         newlist.append("0")
    
for s in ans:
    if "Tie" in s:
        newlist.append("0")
    elif "both" in s and "But" not in s and "but" not in s:
        newlist.append("0")
    elif "Both" in s and "But" not in s and "but" not in s:
        newlist.append("0")
    elif "1" in s or "first" in s:
        newlist.append("1")
    elif "2" in s or "second" in s:
        newlist.append("2")
        
# for s in ans:
#     if "Tie" in s or "both" in s:
#         newlist.append("0")
#     elif "1" in s or "first" in s:
#         newlist.append("1")
#     elif "2" in s or "second" in s:
#         newlist.append("2")
        
ans = newlist
print(len(ans))
print(ans)
print("0" in ans)



999
['1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '1', '2', '2', '2', '1', '1', '1', '1', '1', '2', '0', '0', '2', '2', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '0', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '2', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '0', '0', '1', '0', '2', '1', '1', '0', '0', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '2', '1', '2', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '1', 

In [13]:
correct = 0
category_1 = 0
pred_1 = 0
category_1_correct = 0
category_2 = 0
pred_2 = 0
category_2_correct = 0
category_3 = 0
pred_3 = 0
category_3_correct = 0

for j in range(len(ans)):
  for i in range(3):
    label = pandalm_test_set_prompt[j * 3 + i][1].split("\n\n")[0]
    if label == ans[j]:
        correct += 1
        if ans[j] == "0":
          category_1_correct += 1
        if ans[j] == "1":
          category_2_correct += 1
        if ans[j] == "2":
          category_3_correct += 1
    
    if ans[j] == "0":
      pred_1 += 1
    if ans[j] == "1":
      pred_2 += 1
    if ans[j] == "2":
      pred_3 += 1
      
    if label == "0":
      category_1 += 1
    if label == "1":
      category_2 += 1
    if label == "2":
      category_3 += 1   

print(f"Accuracy: {correct/len(pandalm_test_set_prompt) * 100:.2f}")

def f1_score(p, r):
  return 2 * p * r / (p + r)

p1 = category_1_correct / pred_1
r1 = category_1_correct / category_1
p2 = category_2_correct / pred_2
r2 = category_2_correct / category_2
p3 = category_3_correct / pred_3
r3 = category_3_correct / category_3

print(f"F1: {(f1_score(p1, r1) + f1_score(p2, r2) + f1_score(p3, r3) ) * 100 / 3 :.2f}")

Accuracy: 50.22
F1: 40.16
